In [2]:
import pandas as pd
import json
import geopandas as gpd
import os
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time


import networkx as nx

## Reading JSON file

In [3]:
main_arr = []
with open('D:/Datasets/MIMOS/traffic_flow_10092020_2100_2200.json') as f:
    here_traffic_flow = f.readlines()
    for line in here_traffic_flow:
        
        
        line = json.loads(line)
        line = line['_source']
        #print(line)
        for flow_item_s in line['FIS']:
            for flow_item in flow_item_s['FI']:
                #print(flow_item)
                temp_arr = []
                for tmc_item in flow_item['TMC']:
                    #print(flow_item['TMC'][tmc_item])
                    temp_arr.append(flow_item['TMC'][tmc_item])
                
                ## To get SHP
                #shp_values = flow_item['SHP'][0]['value'][0]
                
                arr_lon = []
                arr_lat = []
                
                fc_temp = flow_item['SHP'][0]['FC']
                
                for shp in flow_item['SHP']:
                    shp_values = shp['value'][0]
                    coordinates_arr = shp_values.split(' ')
                    #print(coordinates_arr)
                    
                    for item in coordinates_arr:
                        if len(item) > 0:
                            item = item.split(',')
                            arr_lat.append(float(item[0]))
                            arr_lon.append(float(item[1]))
                            
                    #if fc_temp != shp['FC']:
                        #print(fc_temp, shp['FC'])
                        #print(line['PBT'])
                        #print(line['DE'])
                        #print('FC Diff')
                    
                polygon_geom = LineString(zip(arr_lon, arr_lat))
                #temp_arr.append(flow_item['SHP'][0]['value'])
                temp_arr.append(polygon_geom)
                
                temp_arr.append(flow_item['SHP'][0]['FC'])
                
                
                ## To get all SHP
                """
                for cf_item in flow_item['SHP'][0]:   
                    temp_arr.append(flow_item['SHP'][0][cf_item])
                
                
                for cf_item in flow_item['CF'][0]:   
                    temp_arr.append(flow_item['CF'][0][cf_item])
                """
                try:
                    temp_arr.append(flow_item['CF'][0]['SP'])
                    temp_arr.append(flow_item['CF'][0]['SU'])
                    temp_arr.append(flow_item['CF'][0]['FF'])
                    temp_arr.append(flow_item['CF'][0]['JF'])
                    temp_arr.append(flow_item['CF'][0]['CN'])
                
                    temp_arr.append(line['PBT'])
                    temp_arr.append(line['DE'])
                
                
                
                    #print(len(temp_arr))
                    if len(temp_arr) == 13:
                        main_arr.append(temp_arr)
                except:
                    pass

In [4]:
here_traffic_flow_df = pd.DataFrame(main_arr)

In [5]:
here_traffic_flow_df.columns = ['TMC_PC', 'TMC_DE', 'TMC_QD', 'TMC_LE',
                            'Geometry', 'FC',
                             'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN',
                               'PBT', 'DE']



In [6]:
here_traffic_flow_df.tail()

,TMC_PC,TMC_DE,TMC_QD,TMC_LE,Geometry,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE
353347,16224,112/Jalan Ayer Hangat,+,0.09207,"LINESTRING (99.8475 6.32254, 99.84769 6.32211,...",3,33.0,33.0,36.0,0.00000,0.7,2020-09-10T21:34:44Z,110
353348,16223,165/Lencongan Putra 3,+,0.35199,"LINESTRING (99.84792 6.32183, 99.84836 6.32134...",3,49.0,49.0,49.7,0.18779,0.7,2020-09-10T21:34:44Z,110
353349,16222,163/Lencongan Putra 1,+,0.87481,"LINESTRING (99.84981999999999 6.31929, 99.8498...",3,43.0,43.0,43.8,0.24353,0.7,2020-09-10T21:34:44Z,110
353350,16221,160/Jalan Beringin,+,0.53555,"LINESTRING (99.85504 6.31359, 99.8551899999999...",3,35.0,35.0,38.4,0.13888,0.7,2020-09-10T21:34:44Z,110
353351,16220,273/Jalan Dato Syed Omar,+,0.61469,"LINESTRING (99.85541000000001 6.30891, 99.8552...",3,34.0,34.0,35.7,0.00000,0.7,2020-09-10T21:34:44Z,110


In [7]:
print(len(here_traffic_flow_df))

353352


## Converting it to GDF

In [8]:
here_traffic_flow_gdf = gpd.GeoDataFrame(here_traffic_flow_df, geometry='Geometry')

## Adding addtional columns

In [9]:
here_traffic_flow_gdf['endpoints'] = here_traffic_flow_gdf['Geometry'].apply(lambda x: x.boundary)

def return_start_point(x):
    #print(x)
    try:
        return x.boundary[0]
    except:
        return None

def return_end_point(x):
    try:
        return x.boundary[1]
    except:
        return None
    

here_traffic_flow_gdf['start_point'] = here_traffic_flow_gdf['Geometry'].apply(lambda x: return_start_point(x))
here_traffic_flow_gdf['end_point'] = here_traffic_flow_gdf['Geometry'].apply(lambda x: return_end_point(x))

In [10]:
here_traffic_flow_gdf.head()

,TMC_PC,TMC_DE,TMC_QD,TMC_LE,Geometry,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE,endpoints,start_point,end_point
0,3725,Bulatan Simpang Lima,+,0.12709,"LINESTRING (101.44326 3.03693, 101.44337 3.036...",2,33.0,33.0,36.4,0.14652,0.7,2020-09-10T21:54:44Z,5,"MULTIPOINT (101.44326 3.03693, 101.44328 3.03613)",POINT (101.44326 3.03693),POINT (101.44328 3.03613)
1,3724,Lebuh Jelutong,+,0.77395,"LINESTRING (101.44328 3.03613, 101.44299 3.035...",2,52.0,52.0,57.1,0.49036,0.7,2020-09-10T21:54:44Z,5,"MULTIPOINT (101.44328 3.03613, 101.44206 3.02964)",POINT (101.44328 3.03613),POINT (101.44206 3.02964)
2,3723,Persiaran Seraya,+,0.72087,"LINESTRING (101.44206 3.02964, 101.44209 3.029...",2,55.0,55.0,55.6,0.14388,0.7,2020-09-10T21:54:44Z,5,"MULTIPOINT (101.44206 3.02964, 101.44279 3.02321)",POINT (101.44206 3.02964),POINT (101.44279 3.02321)
3,3722,Jalan Tun Abdul Razak,+,0.74897,"LINESTRING (101.44279 3.02321, 101.44275 3.023...",2,40.0,40.0,43.2,0.06172,0.7,2020-09-10T21:54:44Z,5,"MULTIPOINT (101.44279 3.02321, 101.44097 3.01669)",POINT (101.44279 3.02321),POINT (101.44097 3.01669)
4,3721,Jalan Tun Sambanthan/B8/Jalan Batu Unjur,+,0.50306,"LINESTRING (101.44097 3.01669, 101.44093 3.016...",2,34.0,34.0,38.4,0.48611,0.7,2020-09-10T21:54:44Z,5,"MULTIPOINT (101.44097 3.01669, 101.44012 3.01227)",POINT (101.44097 3.01669),POINT (101.44012 3.01227)


## Getting Intersection (Only routes within kl n selangor omit if not needed)

In [14]:
gadm = gpd.read_file('D:/Datasets/GeoSpatial/gadm_data/gadm36_MYS_2.shp')
kl_selangor_gadm = gadm[gadm['NAME_1'].isin(['Kuala Lumpur', 'Selangor'])]
print(len(kl_selangor_gadm))
kl_selangor_gadm.head()

10


,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
32,MYS,Malaysia,MYS.4_1,Kuala Lumpur,None,MYS.4.1_1,Kuala Lumpur,None,None,Wilayah Persekutuan,Federal Territory,None,MY.KL.KL,"POLYGON ((101.73227 3.03781, 101.70284 3.03007..."
128,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.1_1,Gombak,None,None,Daerah,District,None,MY.SL.GO,"POLYGON ((101.63429 3.21105, 101.63210 3.20647..."
129,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.2_1,Hulu Langat,Ulu Langat,None,Daerah,District,None,MY.SL.HL,"POLYGON ((101.92247 2.96893, 101.88870 2.89052..."
130,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.3_1,Hulu Selangor,Ulu Selangor,None,Daerah,District,None,MY.SL.HS,"POLYGON ((101.45880 3.39778, 101.46080 3.40378..."
131,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.4_1,Klang,None,None,Daerah,District,None,MY.SL.KE,"MULTIPOLYGON (((101.56800 2.97077, 101.56673 2..."


In [51]:
import geopandas as gpd
#g1 = gpd.GeoDataFrame.from_file("crime_stat.shp")
#g2 = gpd.GeoDataFrame.from_file("population.shp")
data = []
for index, gadm in kl_selangor_gadm.iterrows():
    for index2, traffic in here_traffic_flow_gdf.iterrows():
        if gadm['geometry'].intersects(traffic['Geometry']):
            data.append({'geometry': traffic['Geometry'].intersection(gadm['geometry']),
                        'TMC_PC': traffic['TMC_PC'],
                        'TMC_DE': traffic['TMC_DE'],
                         'TMC_QD': traffic['TMC_QD'],
                         'TMC_LE': traffic['TMC_LE'],
                        'FC': traffic['FC'],
                         'CF_SP': traffic['CF_SP'],
                         'CF_SU': traffic['CF_SU'],
                         'CF_FF': traffic['CF_FF'],
                         'CF_JF': traffic['CF_JF'],
                         'CF_CN': traffic['CF_CN'],
                         'PBT': traffic['PBT'],
                         'DE': traffic['DE'],
                         'endpoints': traffic['endpoints'],
                         'start_point': traffic['start_point'],
                         'end_point': traffic['end_point']
                        })

final_df = gpd.GeoDataFrame(data,columns=['geometry', 'TMC_PC', 'TMC_DE','TMC_QD', 'TMC_LE', 'FC', 
                                    'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN', 'PBT', 'DE', 'endpoints', 'start_point', 'end_point' ])
#df.to_file('intersection.shp')
# control of the results in mi case, first values


In [52]:
final_df.head() # image from a Jupiter/IPython notebook

,geometry,TMC_PC,TMC_DE,TMC_QD,TMC_LE,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE,endpoints,start_point,end_point
0,"LINESTRING (101.74195 3.08229, 101.74197 3.082...",5802,E37/Lebuhraya Hubungan Timur-Barat/Lebuhraya S...,-,0.71360,2,69.0,69.0,70.0,0.20952,0.7,2020-09-10T21:59:44Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"(POINT (101.74195 3.08229), POINT (101.74658 3...",POINT (101.74195 3.08229),POINT (101.74658 3.07899)
1,"LINESTRING (101.74658 3.07899, 101.74688 3.078...",5803,Jalan Cheras Hartamas,-,0.11864,2,70.0,70.0,70.0,0.00000,0.7,2020-09-10T21:59:44Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"(POINT (101.74658 3.07899), POINT (101.74763 3...",POINT (101.74658 3.07899),POINT (101.74763 3.0788)
2,"LINESTRING (101.74763 3.07880, 101.74779 3.078...",5804,Jalan Medan Masria,-,1.14547,2,84.0,84.0,90.0,0.97777,0.7,2020-09-10T21:59:44Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"(POINT (101.74763 3.0788), POINT (101.75745 3....",POINT (101.74763 3.0788),POINT (101.75745 3.07577)
3,"LINESTRING (101.74829 3.07853, 101.74802 3.078...",5803,Jalan Cheras Hartamas,+,0.71240,2,70.0,70.0,84.2,2.47347,0.7,2020-09-10T21:59:44Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"(POINT (101.7533 3.07699), POINT (101.74715 3....",POINT (101.7533 3.07699),POINT (101.74715 3.07869)
4,"LINESTRING (101.74715 3.07869, 101.74685 3.078...",5802,E37/Lebuhraya Hubungan Timur-Barat/Lebuhraya S...,+,0.82177,2,60.0,60.0,67.6,1.64891,0.7,2020-09-10T21:59:44Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"(POINT (101.74715 3.07869), POINT (101.74182 3...",POINT (101.74715 3.07869),POINT (101.74182 3.08248)


In [53]:
final_df.count()

geometry       122628
TMC_PC         122628
TMC_DE         122628
TMC_QD         122628
TMC_LE         122628
FC             122628
CF_SP          122628
CF_SU          122628
CF_FF          122628
CF_JF          122628
CF_CN          122628
PBT            122628
DE             122628
endpoints      122628
start_point    122100
end_point      122100
dtype: int64

## Save as shp

In [14]:
pwd = 'C:\\Users\\tanji\\Desktop\\MIMOS_Geospatial'
try:
    os.mkdir('traffic_flow_shp/traffic_flow_26082020')
except:
    pass
#os.mkdir(pwd + '/traffic_flow/traffic_flow_26082020/')
here_traffic_flow_gdf.to_file('traffic_flow_shp/traffic_flow_26082020/traffic_flow_26082020_0700_0800.shp')

## Save as csv

In [54]:
final_df.to_csv('D:/Datasets/MIMOS/kl_selangor_traffic_flow_traffic_flow_10092020_2100_2200.csv', index=False)

In [10]:
here_traffic_flow_gdf = gpd.GeoDataFrame(pd.read_csv('D:/Datasets/MIMOS/kl_selangor_traffic_flow_traffic_flow_10092020_2100_2200.csv'))
here_traffic_flow_gdf.head()

,geometry,TMC_PC,TMC_DE,TMC_QD,TMC_LE,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE,endpoints,start_point,end_point
0,"LINESTRING (101.74195 3.08229, 101.74197 3.082...",5802,E37/Lebuhraya Hubungan Timur-Barat/Lebuhraya S...,-,0.71360,2,69.0,69.0,70.0,0.20952,0.7,2020-09-10T21:59:44Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"MULTIPOINT (101.74195 3.08229, 101.74658 3.07899)",POINT (101.74195 3.08229),POINT (101.74658 3.07899)
1,"LINESTRING (101.74658 3.07899, 101.74688 3.078...",5803,Jalan Cheras Hartamas,-,0.11864,2,70.0,70.0,70.0,0.00000,0.7,2020-09-10T21:59:44Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"MULTIPOINT (101.74658 3.07899, 101.74763 3.0788)",POINT (101.74658 3.07899),POINT (101.74763 3.0788)
2,"LINESTRING (101.74763 3.0788, 101.74779 3.0787...",5804,Jalan Medan Masria,-,1.14547,2,84.0,84.0,90.0,0.97777,0.7,2020-09-10T21:59:44Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"MULTIPOINT (101.74763 3.0788, 101.75745 3.07577)",POINT (101.74763 3.0788),POINT (101.75745 3.07577)
3,LINESTRING (101.7482891107038 3.07852778655627...,5803,Jalan Cheras Hartamas,+,0.71240,2,70.0,70.0,84.2,2.47347,0.7,2020-09-10T21:59:44Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"MULTIPOINT (101.7533 3.07699, 101.74715 3.07869)",POINT (101.7533 3.07699),POINT (101.74715 3.07869)
4,"LINESTRING (101.74715 3.07869, 101.74685 3.078...",5802,E37/Lebuhraya Hubungan Timur-Barat/Lebuhraya S...,+,0.82177,2,60.0,60.0,67.6,1.64891,0.7,2020-09-10T21:59:44Z,Lebuhraya Cheras-Kajang/Lebuhraya CKE/Cheras-K...,"MULTIPOINT (101.74715 3.07869, 101.74182 3.08248)",POINT (101.74715 3.07869),POINT (101.74182 3.08248)


In [11]:
gadm = gpd.read_file('D:/Datasets/GeoSpatial/gadm_data/gadm36_MYS_2.shp')
kl_selangor_gadm_mys2 = gadm[gadm['NAME_1'].isin(['Kuala Lumpur', 'Selangor'])]
print(len(kl_selangor_gadm_mys2))
kl_selangor_gadm_mys2.head()

10


,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
32,MYS,Malaysia,MYS.4_1,Kuala Lumpur,None,MYS.4.1_1,Kuala Lumpur,None,None,Wilayah Persekutuan,Federal Territory,None,MY.KL.KL,"POLYGON ((101.73227 3.03781, 101.70284 3.03007..."
128,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.1_1,Gombak,None,None,Daerah,District,None,MY.SL.GO,"POLYGON ((101.63429 3.21105, 101.63210 3.20647..."
129,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.2_1,Hulu Langat,Ulu Langat,None,Daerah,District,None,MY.SL.HL,"POLYGON ((101.92247 2.96893, 101.88870 2.89052..."
130,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.3_1,Hulu Selangor,Ulu Selangor,None,Daerah,District,None,MY.SL.HS,"POLYGON ((101.45880 3.39778, 101.46080 3.40378..."
131,MYS,Malaysia,MYS.15_1,Selangor,None,MYS.15.4_1,Klang,None,None,Daerah,District,None,MY.SL.KE,"MULTIPOLYGON (((101.56800 2.97077, 101.56673 2..."


In [12]:
from shapely.geometry import Point, Polygon

data = []
for index2, traffic in here_traffic_flow_gdf.iterrows():
    for index, gadm in kl_selangor_gadm_mys2.iterrows():
        
        try:
            x_start = float(traffic['start_point'].split(' ')[1].replace('(', ''))
            y_start = float(traffic['start_point'].split(' ')[2].replace(')', ''))

            x_end = float(traffic['end_point'].split(' ')[1].replace('(', ''))
            y_end = float(traffic['end_point'].split(' ')[2].replace(')', ''))

            #print(x_start, y_start)
            #print(x_end, y_end)


            if Point(x_start, y_start).within(gadm['geometry']):
                start_district = gadm['NAME_2']
            if Point(x_end, y_end).within(gadm['geometry']):
                end_district = gadm['NAME_2']
        except:
            pass


    data.append({'geometry': traffic['geometry'],
                'TMC_PC': traffic['TMC_PC'],
                'TMC_DE': traffic['TMC_DE'],
                 'TMC_QD': traffic['TMC_QD'],
                 'TMC_LE': traffic['TMC_LE'],
                'FC': traffic['FC'],
                 'CF_SP': traffic['CF_SP'],
                 'CF_SU': traffic['CF_SU'],
                 'CF_FF': traffic['CF_FF'],
                 'CF_JF': traffic['CF_JF'],
                 'CF_CN': traffic['CF_CN'],
                 'PBT': traffic['PBT'],
                 'DE': traffic['DE'],
                 'endpoints': traffic['endpoints'],
                 'start_point': traffic['start_point'],
                 'end_point': traffic['end_point'],
                 'start_district': start_district,
                 'end_district': end_district
                })

final_df_startpoint = gpd.GeoDataFrame(data,columns=['geometry', 'TMC_PC', 'TMC_DE','TMC_QD', 'TMC_LE', 'FC', 
                                    'CF_SP', 'CF_SU', 'CF_FF', 'CF_JF', 'CF_CN', 'PBT', 'DE', 'endpoints', 'start_point', 'end_point', 'start_district', 'end_district'])

In [13]:
final_df_startpoint.tail()

,geometry,TMC_PC,TMC_DE,TMC_QD,TMC_LE,FC,CF_SP,CF_SU,CF_FF,CF_JF,CF_CN,PBT,DE,endpoints,start_point,end_point,start_district,end_district,start_gadm,end_gadm
122623,LINESTRING (101.789786954075 2.868894214316847...,2685,Persiaran Makhota 1/Jalan Mahkota 1/Persiaran ...,-,2.92592,1,107.0,107.0,110.0,0.39999,0.7,2020-09-10T21:44:44Z,Lebuhraya Utara-Selatan,"MULTIPOINT (101.78744 2.85239, 101.78476 2.87658)",POINT (101.78744 2.85239),POINT (101.78476 2.87658),Hulu Langat,Hulu Langat,"POLYGON ((101.9224700927736 2.968930959701652,...","POLYGON ((101.9224700927736 2.968930959701652,..."
122624,"LINESTRING (101.78476 2.87658, 101.78377 2.877...",2686,Persiaran Perkeliling,-,7.68161,1,106.0,106.0,108.3,0.31148,0.7,2020-09-10T21:44:44Z,Lebuhraya Utara-Selatan,"MULTIPOINT (101.78476 2.87658, 101.75578 2.93203)",POINT (101.78476 2.87658),POINT (101.75578 2.93203),Hulu Langat,Hulu Langat,"POLYGON ((101.9224700927736 2.968930959701652,...","POLYGON ((101.9224700927736 2.968930959701652,..."
122625,"LINESTRING (101.75578 2.93203, 101.75572 2.932...",2687,Persiaran Jaya/E18/Lebuhraya Silk,-,6.85666,1,89.0,89.0,90.0,0.16296,0.7,2020-09-10T21:44:44Z,Lebuhraya Utara-Selatan,"MULTIPOINT (101.75578 2.93203, 101.73811 2.99017)",POINT (101.75578 2.93203),POINT (101.73811 2.99017),Hulu Langat,Hulu Langat,"POLYGON ((101.9224700927736 2.968930959701652,...","POLYGON ((101.9224700927736 2.968930959701652,..."
122626,"LINESTRING (101.73811 2.99017, 101.73794 2.990...",2688,Jalan Raya Dua/E9/E18/Lebuhraya Sungai Besi/Le...,-,1.72087,1,89.0,89.0,90.0,0.16296,0.7,2020-09-10T21:44:44Z,Lebuhraya Utara-Selatan,"MULTIPOINT (101.73811 2.99017, 101.72795 3.00155)",POINT (101.73811 2.99017),POINT (101.72795 3.00155),Hulu Langat,Sepang,"POLYGON ((101.9224700927736 2.968930959701652,...","POLYGON ((101.5748977661133 2.971251010894775,..."
122627,"LINESTRING (101.72795 3.00155, 101.72773 3.001...",1434,Lebuhraya Bukit Jalil/Jalan Utama/Persiaran Se...,-,6.60268,1,72.0,72.0,74.8,0.54901,0.7,2020-09-10T21:44:44Z,Lebuhraya Utara-Selatan,"MULTIPOINT (101.72795 3.00155, 101.70498 3.0538)",POINT (101.72795 3.00155),POINT (101.70498 3.0538),Sepang,Kuala Lumpur,"POLYGON ((101.5748977661133 2.971251010894775,...","POLYGON ((101.7322692871095 3.037810325622615,..."


In [14]:
len(final_df_startpoint)

122628

In [15]:
final_df_startpoint.to_csv('D:/Datasets/MIMOS/kl_selangor_traffic_flow_traffic_flow_10092020_2100_2200_districtMovement.csv', index=False)